# Analyse Exploratoire des Données Macroéconomiques

Ce notebook explore les données macroéconomiques collectées via l'API FRED pour identifier les cycles économiques et leurs caractéristiques. Cette analyse servira de base pour notre stratégie de rotation sectorielle.

## Importation des bibliothèques

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings

# Suppression des avertissements pour une meilleure lisibilité
warnings.filterwarnings('ignore')

# Configuration des visualisations
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Ajout du chemin du projet au PYTHONPATH

In [ ]:
# Ajout du répertoire racine du projet au path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)
print(f"Répertoire racine du projet: {project_root}")

## Importation des modules personnalisés

In [ ]:
from src.data.macro_data_collector import MacroDataCollector
from src.models.economic_cycle_classifier import EconomicCycleClassifier

## Collecte et chargement des données

In [ ]:
# Chemin des données prétraitées
data_path = os.path.join(project_root, "data", "processed", "macro_data.csv")

# Vérification si les données existent, sinon les collecter
if not os.path.exists(data_path):
    print("Collecte des données macroéconomiques...")
    # Création du répertoire si nécessaire
    os.makedirs(os.path.dirname(data_path), exist_ok=True)
    
    # Collecte des données (nécessite une clé API FRED)
    collector = MacroDataCollector()
    macro_data = collector.get_all_series(start_date="2000-01-01", frequency='m')
    processed_data = collector.preprocess_data(macro_data)
    
    # Sauvegarde des données
    processed_data.to_csv(data_path)
    print(f"Données sauvegardées dans {data_path}")
else:
    print(f"Chargement des données depuis {data_path}")

# Chargement des données
df = pd.read_csv(data_path, index_col=0, parse_dates=True)
print(f"Données chargées avec succès: {df.shape[0]} observations et {df.shape[1]} variables")

## Aperçu des données

In [ ]:
# Affichage des premières lignes
df.head()

In [ ]:
# Informations sur les colonnes
df.info()

In [ ]:
# Statistiques descriptives
df.describe()

## Vérification des valeurs manquantes

In [ ]:
# Calcul du pourcentage de valeurs manquantes par colonne
missing_values = df.isnull().sum() / len(df) * 100
missing_values = missing_values.sort_values(ascending=False)

# Visualisation des colonnes avec des valeurs manquantes
plt.figure(figsize=(12, 6))
plt.bar(missing_values.index[:15], missing_values.values[:15])
plt.title('Pourcentage de valeurs manquantes par variable')
plt.xlabel('Variables')
plt.ylabel('% de valeurs manquantes')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## Traitement des valeurs manquantes

In [ ]:
# Remplissage des valeurs manquantes avec la méthode forward fill
df_filled = df.fillna(method='ffill')

# Vérification des valeurs manquantes restantes
missing_after = df_filled.isnull().sum().sum()
print(f"Nombre de valeurs manquantes restantes: {missing_after}")

# S'il reste des valeurs manquantes, on utilise backward fill
if missing_after > 0:
    df_filled = df_filled.fillna(method='bfill')
    missing_final = df_filled.isnull().sum().sum()
    print(f"Nombre de valeurs manquantes après bfill: {missing_final}")

# Utilisation du DataFrame traité pour la suite
df = df_filled

## Analyse des indicateurs économiques clés

In [ ]:
# Liste des indicateurs économiques clés à analyser
key_indicators = [
    'GDPC1_YOY',         # Croissance du PIB (annuelle)
    'INDPRO_YOY',        # Croissance de la production industrielle (annuelle)
    'UNRATE',            # Taux de chômage
    'CPIAUCSL_YOY',      # Inflation (annuelle)
    'FEDFUNDS',          # Taux d'intérêt directeur
    'T10Y2Y',            # Spread de taux 10 ans - 2 ans
    'BAMLH0A0HYM2',      # Spread de crédit à haut rendement
    'UMCSENT'            # Confiance des consommateurs
]

# Filtrer pour avoir uniquement les indicateurs disponibles
available_indicators = [indicator for indicator in key_indicators if indicator in df.columns]
print(f"Indicateurs disponibles: {available_indicators}")

### Évolution des indicateurs économiques clés

In [ ]:
# Visualisation de l'évolution des indicateurs économiques clés
fig, axes = plt.subplots(len(available_indicators), 1, figsize=(14, 4*len(available_indicators)))

for i, indicator in enumerate(available_indicators):
    ax = axes[i]
    df[indicator].plot(ax=ax)
    ax.set_title(f'Évolution de {indicator}')
    ax.set_xlabel('Date')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Corrélation entre les indicateurs économiques

In [ ]:
# Calcul de la matrice de corrélation pour les indicateurs clés
corr_matrix = df[available_indicators].corr()

# Visualisation de la matrice de corrélation
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Matrice de corrélation des indicateurs économiques clés')
plt.tight_layout()
plt.show()

## Identification des cycles économiques

### Analyse en composantes principales (ACP)

In [ ]:
# Préparation des données pour l'ACP (standardisation)
X = df[available_indicators].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application de l'ACP
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Création d'un DataFrame pour faciliter la visualisation
pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'], index=df.index)

# Pourcentage de variance expliquée
explained_variance = pca.explained_variance_ratio_ * 100
print(f"Variance expliquée par PC1: {explained_variance[0]:.2f}%")
print(f"Variance expliquée par PC2: {explained_variance[1]:.2f}%")
print(f"Variance totale expliquée: {sum(explained_variance):.2f}%")

In [ ]:
# Visualisation des données dans l'espace de l'ACP
plt.figure(figsize=(12, 8))
plt.scatter(pca_df['PC1'], pca_df['PC2'], alpha=0.7)

# Ajout des dates pour quelques points
for i in range(0, len(pca_df), 24):  # Afficher une date tous les 24 mois
    plt.annotate(pca_df.index[i].strftime('%Y-%m'), 
                 (pca_df['PC1'].iloc[i], pca_df['PC2'].iloc[i]),
                 textcoords="offset points", xytext=(0,10), ha='center')

plt.xlabel(f'PC1 ({explained_variance[0]:.2f}%)')
plt.ylabel(f'PC2 ({explained_variance[1]:.2f}%)')
plt.title('Projection des données macroéconomiques dans l\'espace ACP')
plt.grid(True, alpha=0.3)
plt.show()

### Clustering pour identifier les phases du cycle économique

In [ ]:
# Application du K-means avec 5 clusters (correspondant aux phases du cycle)
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# Ajout des clusters au DataFrame
pca_df['cluster'] = clusters

# Visualisation des clusters dans l'espace de l'ACP
plt.figure(figsize=(12, 8))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['cluster'], cmap='viridis', alpha=0.7)

# Ajout des centres des clusters
centers = pca.transform(kmeans.cluster_centers_)
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.5, marker='X')

plt.xlabel(f'PC1 ({explained_variance[0]:.2f}%)')
plt.ylabel(f'PC2 ({explained_variance[1]:.2f}%)')
plt.title('Identification des phases du cycle économique par clustering')
plt.grid(True, alpha=0.3)
plt.colorbar(scatter, label='Cluster')
plt.show()

### Analyse des caractéristiques des clusters

In [ ]:
# Ajout des clusters au DataFrame original
df['cluster'] = clusters

# Calcul des moyennes par cluster
cluster_means = df.groupby('cluster')[available_indicators].mean()

# Visualisation des moyennes par cluster
plt.figure(figsize=(14, 10))
sns.heatmap(cluster_means, annot=True, cmap='YlGnBu', fmt='.2f', linewidths=0.5)
plt.title('Caractéristiques moyennes des phases du cycle économique')
plt.tight_layout()
plt.show()

### Interprétation des clusters en phases du cycle économique

In [ ]:
# Normalisation des valeurs pour faciliter l'interprétation
normalized_means = pd.DataFrame()
for col in cluster_means.columns:
    normalized_means[col] = (cluster_means[col] - cluster_means[col].min()) / (cluster_means[col].max() - cluster_means[col].min())

# Visualisation des valeurs normalisées
plt.figure(figsize=(14, 10))
sns.heatmap(normalized_means, annot=True, cmap='YlGnBu', fmt='.2f', linewidths=0.5)
plt.title('Caractéristiques normalisées des phases du cycle économique')
plt.tight_layout()
plt.show()

In [ ]:
# Assignation des phases économiques aux clusters
# Cette assignation est basée sur l'analyse des caractéristiques des clusters
# et peut nécessiter une interprétation manuelle

# Exemple d'assignation (à adapter en fonction des résultats)
cluster_phases = {
    0: 'Expansion',    # Forte croissance, faible chômage, inflation modérée
    1: 'Ralentissement', # Croissance en baisse, inflation élevée
    2: 'Récession',    # Croissance négative, chômage élevé
    3: 'Reprise',      # Croissance en hausse, chômage en baisse
    4: 'Surchauffe'    # Croissance forte, inflation élevée
}

# Remplacement des clusters par les phases
df['phase'] = df['cluster'].map(cluster_phases)

# Affichage de la distribution des phases
phase_distribution = df['phase'].value_counts()
print("Distribution des phases économiques:")
print(phase_distribution)

### Visualisation de l'évolution des phases au fil du temps

In [ ]:
# Création d'une variable numérique pour les phases
phase_numeric = df['phase'].map({phase: i for i, phase in enumerate(df['phase'].unique())})

# Visualisation de l'évolution des phases au fil du temps
plt.figure(figsize=(14, 6))
plt.scatter(df.index, phase_numeric, c=phase_numeric, cmap='viridis', s=50, alpha=0.7)

# Ajout des libellés de phases sur l'axe y
phases_unique = sorted(df['phase'].unique())
plt.yticks(range(len(phases_unique)), phases_unique)

plt.title('Évolution des phases du cycle économique au fil du temps')
plt.xlabel('Date')
plt.ylabel('Phase économique')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Utilisation du modèle EconomicCycleClassifier

In [ ]:
# Création du classifieur
classifier = EconomicCycleClassifier(supervised=False)

# Entraînement du classifieur
classifier.fit(df)

# Prédiction des phases
phases = classifier.predict(df)

# Visualisation de la distribution des phases
fig = classifier.plot_cycle_distribution(df)
plt.show()

# Visualisation des caractéristiques des phases
fig = classifier.plot_cycle_characteristics(df)
plt.show()

## Analyse des récessions officielles (NBER)

In [ ]:
# Définition des périodes de récession officielles selon le NBER
recessions = [
    ('2001-03-01', '2001-11-01'),  # Éclatement de la bulle internet
    ('2007-12-01', '2009-06-01'),  # Grande récession
    ('2020-02-01', '2020-04-01')   # Pandémie COVID-19
]

# Fonction pour ajouter les zones de récession à un graphique
def add_recession_bars(ax):
    for start, end in recessions:
        ax.axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.2, color='red')

# Visualisation des indicateurs clés avec les périodes de récession
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# PIB
ax = axes[0]
if 'GDPC1_YOY' in df.columns:
    df['GDPC1_YOY'].plot(ax=ax)
    ax.set_title('Croissance du PIB (en %)')
    ax.set_xlabel('')
    ax.grid(True, alpha=0.3)
    add_recession_bars(ax)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)

# Chômage
ax = axes[1]
if 'UNRATE' in df.columns:
    df['UNRATE'].plot(ax=ax)
    ax.set_title('Taux de chômage (en %)')
    ax.set_xlabel('')
    ax.grid(True, alpha=0.3)
    add_recession_bars(ax)

# Inflation
ax = axes[2]
if 'CPIAUCSL_YOY' in df.columns:
    df['CPIAUCSL_YOY'].plot(ax=ax)
    ax.set_title('Inflation (en %)')
    ax.grid(True, alpha=0.3)
    add_recession_bars(ax)
    ax.axhline(y=2, color='k', linestyle='--', alpha=0.3)  # Cible d'inflation typique

plt.tight_layout()
plt.show()

## Comparaison des phases identifiées avec les récessions officielles

In [ ]:
# Création d'une colonne pour les récessions officielles
df['official_recession'] = 0

# Marquage des périodes de récession
for start, end in recessions:
    mask = (df.index >= pd.Timestamp(start)) & (df.index <= pd.Timestamp(end))
    df.loc[mask, 'official_recession'] = 1

# Visualisation de la concordance entre les phases identifiées et les récessions officielles
plt.figure(figsize=(14, 6))

# Affichage des phases identifiées
plt.scatter(df.index, phase_numeric, c=phase_numeric, cmap='viridis', s=50, alpha=0.7, label='Phases identifiées')

# Affichage des récessions officielles
for start, end in recessions:
    plt.axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.2, color='red', label='_nolegend_')

# Ajout des libellés de phases sur l'axe y
plt.yticks(range(len(phases_unique)), phases_unique)
plt.title('Comparaison des phases identifiées avec les récessions officielles')
plt.xlabel('Date')
plt.ylabel('Phase économique')
plt.grid(True, alpha=0.3)

# Ajout d'une légende pour les récessions
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='red', alpha=0.2, label='Récession officielle (NBER)')]
plt.legend(handles=legend_elements)

plt.tight_layout()
plt.show()

## Préparation pour l'analyse sectorielle

Maintenant que nous avons identifié les phases du cycle économique, nous pourrons explorer dans le prochain notebook comment les différents secteurs se comportent dans chaque phase.

In [ ]:
# Sauvegarde des phases identifiées pour utilisation dans d'autres notebooks
phases_path = os.path.join(project_root, "data", "processed", "economic_phases.csv")
phases_df = pd.DataFrame({'date': df.index, 'phase': df['phase']})
phases_df.to_csv(phases_path, index=False)
print(f"Phases économiques sauvegardées dans {phases_path}")

## Sauvegarde du modèle entraîné

In [ ]:
# Sauvegarde du modèle pour utilisation dans d'autres notebooks
model_path = os.path.join(project_root, "models", "economic_cycle_classifier.joblib")
os.makedirs(os.path.dirname(model_path), exist_ok=True)
classifier.save_model(model_path)
print(f"Modèle de classification des cycles économiques sauvegardé dans {model_path}")

## Conclusion

Dans ce notebook, nous avons exploré les données macroéconomiques et identifié les phases du cycle économique. Nous avons constaté que :

1. Les indicateurs économiques (PIB, chômage, inflation, etc.) montrent des tendances claires qui peuvent être utilisées pour identifier les phases du cycle.
2. Le clustering nous a permis d'identifier 5 phases distinctes du cycle économique : Expansion, Surchauffe, Ralentissement, Récession et Reprise.
3. Ces phases correspondent généralement bien aux récessions officielles identifiées par le NBER.
4. Notre modèle de classification des cycles économiques peut être utilisé pour prédire la phase actuelle du cycle et guider notre stratégie de rotation sectorielle.

Dans le prochain notebook, nous analyserons les performances sectorielles dans chaque phase du cycle économique pour identifier les opportunités de rotation sectorielle.